In [1]:
import numpy as np
import galois
from py_ecc.optimized_bls12_381 import add, multiply, G1, G2, neg, pairing, eq, normalize

In [2]:
class SRS:
    def __init__(self, tau, n = 2):
        self.tau = tau
        self.tau1 = [multiply(G1, int(tau)**i) for i in range(0, n + 3)]
        self.tau2 = multiply(G2, int(tau))

    def __str__(self):
        s = f"tau: {self.tau}\n"
        s += "".join([f"[tau^{i}]G1: {str(normalize(point))}\n" for i, point in enumerate(self.tau1)])
        s += f"[tau^{i}]G2: {str(normalize(self.tau2))}\n"
        return s


def new_call(self, at, **kwargs):
    if isinstance(at, SRS):
        coeffs = self.coeffs[::-1]
        result = multiply(at.tau1[0], coeffs[0])
        for i in range(1, len(coeffs)):
            result = add(result, multiply(at.tau1[i], coeffs[i]))
        return result

    return galois.Poly.original_call(self, at, **kwargs)

In [3]:
galois.Poly.original_call = galois.Poly.__call__
galois.Poly.__call__ = new_call

In [4]:
x = 2
y = 3

# 2x^2 - x^2y^2 + 3
out = 2*x**2 - x**2*y**2 + 3
print(f"out = {out}")

# We have 7 gates, next power of 2 is 8
n = 7
n = 2**int(np.ceil(np.log2(n)))
assert n & n - 1 == 0, "n must be a power of 2"

# Prime field p
p = 241
Fp = galois.GF(p)

# Find primitive root of unity
omega = Fp.primitive_root_of_unity(n)
assert omega**(n) == 1, f"omega (ω) {omega} is not a root of unity"

roots = Fp([omega**i for i in range(n)])
print(f"roots = {roots}")

out = -25
roots = [  1  30 177   8 240 211  64 233]


In [5]:
def pad_array(a, n):
    return a + [0]*(n - len(a))

# witness vectors
a = [2, 2, 3, 4, 4, 8, -28]
b = [2, 2, 3, 0, 9, 36, 3]
c = [4, 4, 9, 8, 36, -28, -25]

# gate vectors
ql = [0, 0, 0, 2, 0, 1, 1]
qr = [0, 0, 0, 0, 0, -1, 0]
qm = [1, 1, 1, 1, 1, 0, 0]
qc = [0, 0, 0, 0, 0, 0, 3]
qo = [-1, -1, -1, -1, -1, -1, -1]
qpi = [0, 0, 0, 0, 0, 0, 0]

# pad vectors to length n
a = pad_array(a, n)
b = pad_array(b, n)
c = pad_array(c, n)
ql = pad_array(ql, n)
qr = pad_array(qr, n)
qm = pad_array(qm, n)
qc = pad_array(qc, n)
qo = pad_array(qo, n)
qpi = pad_array(qpi, n)

print(f"a = {a}")
print(f"b = {b}")
print(f"c = {c}")
print(f"ql = {ql}")
print(f"qr = {qr}")
print(f"qm = {qm}")
print(f"qc = {qc}")
print(f"qo = {qo}")
print(f"qpi = {qpi}")

a = [2, 2, 3, 4, 4, 8, -28, 0]
b = [2, 2, 3, 0, 9, 36, 3, 0]
c = [4, 4, 9, 8, 36, -28, -25, 0]
ql = [0, 0, 0, 2, 0, 1, 1, 0]
qr = [0, 0, 0, 0, 0, -1, 0, 0]
qm = [1, 1, 1, 1, 1, 0, 0, 0]
qc = [0, 0, 0, 0, 0, 0, 3, 0]
qo = [-1, -1, -1, -1, -1, -1, -1, 0]
qpi = [0, 0, 0, 0, 0, 0, 0, 0]


In [6]:
def print_sigma(sigma, a, b, c, r):
    group_size = len(sigma) // 3
    padding = 6

    print(f"{' w'} | {'value':{padding}} | {'i':{padding}} | {'sigma(i)':{padding}}")

    for i in range(0, group_size):
        print(f"a{i} | {a[i]:{padding}} | {r[i]:{padding}} | {r[sigma[i]]:{padding}}")

    print(f"-- | {'--':{padding}} | {'--':{padding}} | {'--':{padding}}")

    for i in range(group_size, 2 * group_size):
        print(f"b{i - group_size} | {b[i - group_size]:{padding}} | {r[i]:{padding}} | {r[sigma[i]]:{padding}}")

    print(f"-- | {'--':{padding}} | {'--':{padding}} | {'--':{padding}}")

    for i in range(2 * group_size, 3 * group_size):
        print(f"c{i - 2 * group_size} | {c[i - 2 * group_size]:{padding}} | {r[i]:{padding}} | {r[sigma[i]]:{padding}}")

ai = range(0, n)
bi = range(n, 2*n)
ci = range(2*n, 3*n)

sigma = {
    ai[0]: ai[0], ai[1]: ai[1], ai[2]: ai[2], ai[3]: ci[0], ai[4]: ci[1], ai[5]: ci[3], ai[6]: ci[5], ai[7]: ai[7],
    bi[0]: bi[0], bi[1]: bi[1], bi[2]: bi[2], bi[3]: bi[3], bi[4]: ci[2], bi[5]: ci[4], bi[6]: bi[6], bi[7]: bi[7],
    ci[0]: ai[3], ci[1]: ai[4], ci[2]: bi[4], ci[3]: ai[5], ci[4]: bi[5], ci[5]: ai[6], ci[6]: ci[6], ci[7]: ci[7],
}

k1 = 2
k2 = 4
c1_roots = roots
c2_roots = roots * k1
c3_roots = roots * k2

c_roots = np.concatenate((c1_roots, c2_roots, c3_roots))

check = set()
for r in c_roots:
    assert not int(r) in check, f"Duplicate root {r} in {c_roots}"
    check.add(int(r))

sigma1 = Fp([c_roots[sigma[i]] for i in range(0, n)])
sigma2 = Fp([c_roots[sigma[i + n]] for i in range(0, n)])
sigma3 = Fp([c_roots[sigma[i + 2 * n]] for i in range(0, n)])

print_sigma(sigma, a, b, c, c_roots)

print("\n\n--- Cosest ---")
print(f"c0 = {c1_roots}")
print(f"c1 = {c2_roots}")
print(f"c2 = {c3_roots}")

print("\n\n--- Sigma ---")
print(f"sigma1 = {sigma1}")
print(f"sigma2 = {sigma2}")
print(f"sigma3 = {sigma3}")

 w | value  | i      | sigma(i)
a0 |      2 |      1 |      1
a1 |      2 |     30 |     30
a2 |      3 |    177 |    177
a3 |      4 |      8 |      4
a4 |      4 |    240 |    120
a5 |      8 |    211 |     32
a6 |    -28 |     64 |    121
a7 |      0 |    233 |    233
-- | --     | --     | --    
b0 |      2 |      2 |      2
b1 |      2 |     60 |     60
b2 |      3 |    113 |    113
b3 |      0 |     16 |     16
b4 |      9 |    239 |    226
b5 |     36 |    181 |    237
b6 |      3 |    128 |    128
b7 |      0 |    225 |    225
-- | --     | --     | --    
c0 |      4 |      4 |      8
c1 |      4 |    120 |    240
c2 |      9 |    226 |    239
c3 |      8 |     32 |    211
c4 |     36 |    237 |    181
c5 |    -28 |    121 |     64
c6 |    -25 |     15 |     15
c7 |      0 |    209 |    209


--- Cosest ---
c0 = [  1  30 177   8 240 211  64 233]
c1 = [  2  60 113  16 239 181 128 225]
c2 = [  4 120 226  32 237 121  15 209]


--- Sigma ---
sigma1 = [  1  30 177   4 120  32 121 

In [7]:
def to_galois_array(vector, field):
    # normalize to positive values
    a = [x % field.order for x in vector]
    return field(a)

def to_poly(x, v, field):
    assert len(x) == len(v)
    y = to_galois_array(v, field) if type(v) == list else v
    return galois.lagrange_poly(x, y)

QL = to_poly(roots, ql, Fp)
QR = to_poly(roots, qr, Fp)
QM = to_poly(roots, qm, Fp)
QC = to_poly(roots, qc, Fp)
QO = to_poly(roots, qo, Fp)
QPI = to_poly(roots, qpi, Fp)

print("--- Gate Polynomials ---")
print(f"QL = {QL}")
print(f"QR = {QR}")
print(f"QM = {QM}")
print(f"QC = {QC}")
print(f"QO = {QO}")
print(f"QPI = {QPI}")

--- Gate Polynomials ---
QL = 187x^7 + 38x^6 + 119x^5 + 60x^4 + 70x^3 + 22x^2 + 106x + 121
QR = 64x^7 + 8x^6 + x^5 + 211x^4 + 177x^3 + 233x^2 + 240x + 30
QM = 57x^7 + 211x^6 + 73x^5 + 211x^4 + 168x^3 + 211x^2 + 184x + 91
QC = 24x^7 + 90x^6 + 217x^5 + 151x^4 + 24x^3 + 90x^2 + 217x + 151
QO = 240x^7 + 8x^6 + 177x^5 + 30x^4 + x^3 + 233x^2 + 64x + 210
QPI = 0


In [8]:
def to_vanishing_poly(roots, field):
    # Z^n - 1 = (Z - 1)(Z - w)(Z - w^2)...(Z - w^(n-1))
    return galois.Poly.Degrees([len(roots), 0], coeffs=[1, -1], field=field)

Zh = to_vanishing_poly(roots, Fp)
for x in roots:
    assert Zh(x) == 0

print("--- Vanishing Polynomial ---")
print(f"Zh = {Zh}")

--- Vanishing Polynomial ---
Zh = x^8 + 240


In [9]:
S1 = to_poly(roots, sigma1, Fp)
S2 = to_poly(roots, sigma2, Fp)
S3 = to_poly(roots, sigma3, Fp)

I1 = to_poly(roots, c1_roots, Fp)
I2 = to_poly(roots, c2_roots, Fp)
I3 = to_poly(roots, c3_roots, Fp)

padding = 3
for i in range(0, len(roots)):
    s  = f"i = {i:{padding}} --> {roots[i]:{padding}} "
    s += f"   I1({roots[i]:{padding}}) = {I1(roots[i]):{padding}} "
    s += f"   I2({roots[i]:{padding}}) = {I2(roots[i]):{padding}} "
    s += f"   I3({roots[i]:{padding}}) = {I3(roots[i]):{padding}} "
    s += f"   S1({roots[i]:{padding}}) = {S1(roots[i]):{padding}} "
    s += f"   S2({roots[i]:{padding}}) = {S2(roots[i]):{padding}} "
    s += f"   S3({roots[i]:{padding}}) = {S3(roots[i]):{padding}} "
    print(s)

    assert I1(roots[i]) == roots[i], f"I1({roots[i]}) != {roots[i]}"
    assert I2(roots[i]) == k1 * roots[i], f"I2({roots[i]}) != {k1 * roots[i]}"
    assert I3(roots[i]) == k2 * roots[i], f"I3({roots[i]}) != {k2 * roots[i]}"

    assert S1(roots[i]) == sigma1[i], f"S1({roots[i]}) != {sigma1[i]}"
    assert S2(roots[i]) == sigma2[i], f"S2({roots[i]}) != {sigma2[i]}"
    assert S3(roots[i]) == sigma3[i], f"S3({roots[i]}) != {sigma3[i]}"

i =   0 -->   1    I1(  1) =   1    I2(  1) =   2    I3(  1) =   4    S1(  1) =   1    S2(  1) =   2    S3(  1) =   8 
i =   1 -->  30    I1( 30) =  30    I2( 30) =  60    I3( 30) = 120    S1( 30) =  30    S2( 30) =  60    S3( 30) = 240 
i =   2 --> 177    I1(177) = 177    I2(177) = 113    I3(177) = 226    S1(177) = 177    S2(177) = 113    S3(177) = 239 
i =   3 -->   8    I1(  8) =   8    I2(  8) =  16    I3(  8) =  32    S1(  8) =   4    S2(  8) =  16    S3(  8) = 211 
i =   4 --> 240    I1(240) = 240    I2(240) = 239    I3(240) = 237    S1(240) = 120    S2(240) = 226    S3(240) = 181 
i =   5 --> 211    I1(211) = 211    I2(211) = 181    I3(211) = 121    S1(211) =  32    S2(211) = 237    S3(211) =  64 
i =   6 -->  64    I1( 64) =  64    I2( 64) = 128    I3( 64) =  15    S1( 64) = 121    S2( 64) = 128    S3( 64) =  15 
i =   7 --> 233    I1(233) = 233    I2(233) = 225    I3(233) = 209    S1(233) = 233    S2(233) = 225    S3(233) = 209 


In [10]:
def generate_tau(encrypted=False):
    return SRS(Fp.Random(), n) if encrypted else Fp.Random()

tau = generate_tau()
print(f"--- Tau ---")
print(tau)

--- Tau ---
232


In [11]:
random_b = [Fp.Random() for i in range(0, 9)]

bA = galois.Poly(random_b[:2], field=Fp)
bB = galois.Poly(random_b[2:4], field=Fp)
bC = galois.Poly(random_b[4:6], field=Fp)

_A = to_poly(roots, a, Fp)
_B = to_poly(roots, b, Fp)
_C = to_poly(roots, c, Fp)

A = _A + bA*Zh
B = _B + bB*Zh
C = _C + bC*Zh

# gate constraints polynomial
# g(x) = a(x)*ql(x) + b(x)*qr(x) + a(x)*b(x)*qm(x) + qc(x) + c(x)*qo(x)
G = A*QL + B*QR + A*B*QM + QC + C*QO + QPI

print("--- Gate Constraints Polynomial ---")
print(f"G = {G}")
for i in range(0, len(roots)):
    print(f"gate #{i} G({roots[i]}) = {G(roots[i])} --> {'OK' if G(roots[i]) == 0 else 'FAIL'}")
    assert G(roots[i]) == 0, f"G({roots[i]}) != 0"

assert G % Zh == 0, f"G(x) % Zh(x) != 0"

padding = 3
for i in range(0, len(roots)):
    s = f"i = {i:{padding}} --> {roots[i]:{padding}} "
    s += f"   A({roots[i]:{padding}}) = {A(roots[i]):{padding}} "
    s += f"   B({roots[i]:{padding}}) = {B(roots[i]):{padding}} "
    s += f"   C({roots[i]:{padding}}) = {C(roots[i]):{padding}} "
    print(s)

round1 = [A(tau), B(tau), C(tau)]
print("\n\n--- Round 1 ---")
print(f"Round 1 = {round1}")
print("TODO: [A], [B], [C]")

--- Gate Constraints Polynomial ---
G = 209x^25 + 131x^24 + 89x^23 + 203x^22 + 112x^21 + x^20 + 89x^19 + 34x^18 + 107x^17 + 175x^16 + 47x^15 + 83x^14 + 163x^13 + 122x^12 + 160x^11 + 10x^10 + 32x^9 + 26x^8 + 105x^7 + 196x^6 + 207x^5 + 118x^4 + 233x^3 + 197x^2 + 134x + 150
gate #0 G(1) = 0 --> OK
gate #1 G(30) = 0 --> OK
gate #2 G(177) = 0 --> OK
gate #3 G(8) = 0 --> OK
gate #4 G(240) = 0 --> OK
gate #5 G(211) = 0 --> OK
gate #6 G(64) = 0 --> OK
gate #7 G(233) = 0 --> OK
i =   0 -->   1    A(  1) =   2    B(  1) =   2    C(  1) =   4 
i =   1 -->  30    A( 30) =   2    B( 30) =   2    C( 30) =   4 
i =   2 --> 177    A(177) =   3    B(177) =   3    C(177) =   9 
i =   3 -->   8    A(  8) =   4    B(  8) =   0    C(  8) =   8 
i =   4 --> 240    A(240) =   4    B(240) =   9    C(240) =  36 
i =   5 --> 211    A(211) =   8    B(211) =  36    C(211) = 213 
i =   6 -->  64    A( 64) = 213    B( 64) =   3    C( 64) = 216 
i =   7 --> 233    A(233) =   0    B(233) =   0    C(233) =   0 


--- 

In [15]:
from Crypto.Hash import keccak

def numbers_to_hash(numbers: [int], field) -> int:
    """Hash a number."""
    s = keccak.new(digest_bits=256)
    s.update(bytes(numbers))    
    return field(int(s.hexdigest(), 16) % field.order)


# TODO: calculate beta, gamma from [A], [B], [C]
beta = numbers_to_hash(round1 + [0], Fp)
gamma = numbers_to_hash(round1 + [1], Fp)

_F = (A + I1 * beta + gamma) * (B + I2 * beta + gamma) * (C + I3 * beta + gamma)
_G = (A + S1 * beta + gamma) * (B + S2 * beta + gamma) * (C + S3 * beta + gamma)

acc_eval = [Fp(1)]
for i in range(0, n):
    acc_eval.append(
        acc_eval[-1] * (_F(roots[i]) / _G(roots[i]))
    )
assert acc_eval.pop() == Fp(1)
ACC = galois.lagrange_poly(roots, Fp(acc_eval))
print("\n\n--- Accumulator Polynomial ---")
print(f"ACC(x) = {ACC}")

bZ = galois.Poly(random_b[6:9], field=Fp)
print(f"bZ = {bZ}")
Z = bZ * Zh + ACC

print("\n\n--- Z Polynomial ---")
print(f"Z(x) = {Z}")

for r in roots:
    print(f"Z({r}) = {Z(r)}")

assert Z(roots[0]) == 1
assert Z(roots[-1]) == 1

round2 = [Z(tau)]
print("\n\n--- Round 2 ---")
print(f"Round 2 = {round2}")
print("TODO: [Z]")



--- Accumulator Polynomial ---
ACC(x) = 210x^7 + 92x^6 + 207x^5 + 233x^4 + 221x^3 + 231x^2 + 40x + 213
bZ = 185x^2 + 129x + 82


--- Z Polynomial ---
Z(x) = 185x^10 + 129x^9 + 82x^8 + 210x^7 + 92x^6 + 207x^5 + 233x^4 + 221x^3 + 46x^2 + 152x + 131
Z(1) = 1
Z(30) = 140
Z(177) = 90
Z(8) = 158
Z(240) = 91
Z(211) = 103
Z(64) = 156
Z(233) = 1


--- Round 2 ---
Round 2 = [GF(204, order=241)]
TODO: [Z]


In [16]:
def shift_poly(poly: galois.Poly, omega: Fp):
    coeffs = poly.coeffs[::-1]
    coeffs = [c * omega**i for i, c in enumerate(coeffs)]
    return galois.Poly(coeffs[::-1], field=poly.field)

alpha = numbers_to_hash(round1 + round2, Fp)

Zomega = shift_poly(Z, omega=omega)
for r in roots:
    print(f"Z({r:3}) = {Z(r):3} -> Zω({r:3}) = {Zomega(r):3}")

print("\n\n--- Zω Polynomial ---")
print(f"Z(ωx) = {Zomega}")

L1 = galois.lagrange_poly(roots, Fp([1] + [Fp(0)] * (n - 1)))

print("\n\n--- L1 Polynomial ---")
print(f"L1(x) = {L1}")
for i, r in enumerate(roots):
    print(f"L1({r}) = {L1(r)}")
    assert L1(r) == (Fp(1) if i == 0 else Fp(0))

T0 = G
assert T0 % Zh == 0, f"T0(x) % Zh(x) != 0"

T1 = (_F * Z - _G * Zomega) * alpha
assert T1 % Zh == 0, f"T1(x) % Zh(x) != 0"

T2 = (Z - galois.Poly([1], field=Fp)) * L1 * alpha**2
assert T2 % Zh == 0, f"T2(x) % Zh(x) != 0"

T = (T0 + T1 + T2)
assert T % Zh == 0, f"T(x) % Zh(x) != 0"

for r in roots:
    assert T(r) == 0, f"T({r}) != 0"

T = T // Zh

print("\n\n--- T Polynomial ---")
print(f"T(x) = {T}")

t_coeffs = T.coeffs[::-1]

Tl = galois.Poly(t_coeffs[:n+2][::-1], field=Fp)
Tm = galois.Poly(t_coeffs[n+2:2*(n+2)][::-1], field=Fp)
Th = galois.Poly(t_coeffs[2*(n+2):][::-1], field=Fp)

X_n = galois.Poly.Degrees([n+2, 0], coeffs=[1, 0], field=Fp)
X_2n = galois.Poly.Degrees([2*(n+2), 0], coeffs=[1, 0], field=Fp)
# make sure that T was split correctly
# T = TL + X^n * TM + X^2n * TH
assert T == (Tl + X_n * Tm + X_2n * Th)
assert T.degree == 3 * n + 5

print("\n\n--- T' ---")
print(f"Tl(x) = {Tl}")
print(f"Tm(x) = {Tm}")
print(f"Th(x) = {Th}")

round3 = [Tl(tau), Tm(tau), Th(tau)]
print("\n\n--- Round 3 ---")
print(f"Round 3 = {round3}")
print("TODO: [Tl], [Tm], [Th]")

Z(  1) =   1 -> Zω(  1) = 140
Z( 30) = 140 -> Zω( 30) =  90
Z(177) =  90 -> Zω(177) = 158
Z(  8) = 158 -> Zω(  8) =  91
Z(240) =  91 -> Zω(240) = 103
Z(211) = 103 -> Zω(211) = 156
Z( 64) = 156 -> Zω( 64) =   1
Z(233) =   1 -> Zω(233) =   1


--- Zω Polynomial ---
Z(ωx) = 210x^10 + 14x^9 + 82x^8 + 7x^7 + 104x^6 + 56x^5 + 8x^4 + 81x^3 + 189x^2 + 222x + 131


--- L1 Polynomial ---
L1(x) = 211x^7 + 211x^6 + 211x^5 + 211x^4 + 211x^3 + 211x^2 + 211x + 211
L1(1) = 1
L1(30) = 0
L1(177) = 0
L1(8) = 0
L1(240) = 0
L1(211) = 0
L1(64) = 0
L1(233) = 0


--- T Polynomial ---
T(x) = 146x^29 + 230x^28 + 100x^27 + 68x^26 + 145x^25 + 167x^24 + 218x^23 + 238x^22 + 111x^21 + 110x^20 + 77x^19 + 88x^18 + 234x^17 + 4x^16 + 217x^15 + 219x^14 + 165x^13 + 47x^12 + 14x^11 + 213x^10 + 174x^9 + 90x^8 + 66x^7 + 44x^6 + 26x^5 + 69x^4 + 60x^3 + 37x^2 + 4x + 126


--- T' ---
Tl(x) = 174x^9 + 90x^8 + 66x^7 + 44x^6 + 26x^5 + 69x^4 + 60x^3 + 37x^2 + 4x + 126
Tm(x) = 77x^9 + 88x^8 + 234x^7 + 4x^6 + 217x^5 + 219x^4 + 165x^3

In [17]:
zeta = numbers_to_hash(round1 + round2 + round3, Fp)

a_zeta = A(zeta)
b_zeta = B(zeta)
c_zeta = C(zeta)
s1_zeta = S1(zeta)
s2_zeta = S2(zeta)
t_zeta = T(zeta)
z_omega_zeta = Zomega(zeta)

R = (QM * a_zeta * b_zeta +
    QL * a_zeta +
    QR * b_zeta +
    QO * c_zeta +
    QC)
R += (Z * 
    (a_zeta + beta * zeta + gamma) *
    (b_zeta + beta * zeta * k1 + gamma) *
    (c_zeta + beta * zeta * k2 + gamma) * alpha)
R -= (S3 *
    (a_zeta + beta * s1_zeta + gamma) *
    (b_zeta + beta * s2_zeta + gamma) * alpha * beta * z_omega_zeta)
R += Z * L1(zeta) * alpha**2

print("\n\n--- R ---")
print(f"R(x) = {R}")

r_zeta = R(zeta)

round4 = [a_zeta, b_zeta, c_zeta, s1_zeta, s2_zeta, t_zeta, z_omega_zeta, r_zeta]
print("\n\n--- Round 4 ---")
print(f"Round 4 = {round4}")



--- R ---
R(x) = 98x^10 + 196x^9 + 218x^8 + 227x^7 + 44x^6 + 128x^5 + 110x^4 + 193x^3 + 229x^2 + 130x + 48


--- Round 4 ---
Round 4 = [GF(47, order=241), GF(1, order=241), GF(68, order=241), GF(23, order=241), GF(140, order=241), GF(106, order=241), GF(130, order=241), GF(166, order=241)]


In [18]:
v = numbers_to_hash(round1 + round2 + round3 + round4, Fp)

X_minus_zeta = galois.Poly([1, -zeta], field=Fp)
print(f"X - zeta = {X_minus_zeta}")

Wzeta = (Tl + zeta**(n+2) * Tm + zeta**(2*(n+2)) * Th - t_zeta) + \
        (R - r_zeta) * v + \
        (A - a_zeta) * v**2 + \
        (B - b_zeta) * v**3 + \
        (C - c_zeta) * v**4 + \
        (S1 - s1_zeta) * v**5 + \
        (S2 - s2_zeta) * v**6

assert Wzeta % X_minus_zeta == 0, f"Wzeta(x) % X - zeta != 0"
Wzeta = Wzeta // X_minus_zeta

X_minus_omega_zeta = galois.Poly([1, -(omega*zeta)], field=Fp)
print(f"X - ω*zeta = {X_minus_omega_zeta}")

Womega_zeta = (Z - z_omega_zeta)
assert Womega_zeta % X_minus_omega_zeta == 0, f"Womega_zeta(x) % X - ω*zeta != 0"
Womega_zeta = Womega_zeta // X_minus_omega_zeta

round5 = [Wzeta(tau), Womega_zeta(tau)]
print("\n\n--- Round 5 ---")
print(f"Round 5 = {round5}")
print("TODO: [Wzeta], [Womega_zeta]")

u = numbers_to_hash(round1 + round2 + round3 + round4 + round5, Fp)

X - zeta = x + 166
X - ω*zeta = x + 160


--- Round 5 ---
Round 5 = [GF(63, order=241), GF(117, order=241)]
TODO: [Wzeta], [Womega_zeta]


In [19]:
qm_exp = QM(tau)
ql_exp = QL(tau)
qr_exp = QR(tau)
qo_exp = QO(tau)
qc_exp = QC(tau)
qpi_exp = QPI(tau)
z_exp = Z(tau)
s1_exp = S1(tau)
s2_exp = S2(tau)
s3_exp = S3(tau)
tl_exp = Tl(tau)
tm_exp = Tm(tau)
th_exp = Th(tau)

a_exp = A(tau)
b_exp = B(tau)
c_exp = C(tau)

w_zeta_exp = Wzeta(tau)
w_omega_zeta_exp = Womega_zeta(tau)

In [20]:
Zh_z = Zh(zeta)
L1_z = L1(zeta)
t = r_zeta - ((a_zeta + beta * s1_zeta + gamma) * (b_zeta + beta * s2_zeta + gamma) * (c_zeta + gamma) * z_omega_zeta) * alpha - L1_z * alpha**2
t /= Zh_z

assert t == t_zeta

In [21]:
D_exp = (qm_exp * a_zeta * b_zeta * v +
        ql_exp * a_zeta * v +
        qr_exp * b_zeta * v +
        qo_exp * c_zeta * v +
        qc_exp * v)

D_exp += (z_exp * (
        (a_zeta + beta * zeta + gamma) *
        (b_zeta + beta * zeta * k1 + gamma) *
        (c_zeta + beta * zeta * k2 + gamma) * alpha * v
        + L1_z * alpha**2 * v + u))

D_exp -= (s3_exp *
        (a_zeta + beta * s1_zeta + gamma) *
        (b_zeta + beta * s2_zeta + gamma) * 
        alpha * beta * z_omega_zeta * v)